Первичная чистка данных, убираю лишний текст, очищаю ссылки на соц.сети

In [23]:
#Загружаю нужные библиотеки

import pandas as pd

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# для запросов
import urllib3

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [24]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [25]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [26]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
#opt.add_argument('--proxy-server=193.32.191.161:45785') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [27]:
# настройки urllib

http = urllib3.PoolManager()
headers = urllib3.make_headers(keep_alive=True, user_agent=user_agent)

In [28]:
# запускаю драйвер, проверяю настройки

#driver = webdriver.Chrome(options=opt)
#driver.get(test_url)

In [29]:
work = pd.read_excel('C:/Data/data_stom.xlsx', sheet_name='data')

In [30]:
work.head(3)

,page_url,object_name,address,phone,hours,website,social_network,fb_link,inst_link,vk_link,...,clinic_rating,review_number,description,clinic_info,doc_pics,specialists_dict,price,services,contacts,soc_links
0,https://zoon.ru/msk/medical/stomatologicheskay...,Ника Орто,"Москва, к1126, Зеленоград",89660326559,"Пн-Чт, Сб-Вс: 10:00—20:00 и до последнего клие...",none,none,none,none,none,...,4.4,21 отзыв,"Гордиться улыбкой, без стеснения дарить ее окр...","['Юридическое название: ООО ""Ника Денталь""']",3,{'Мадина Болатовна Сатыбалова ': ['Стаж 14 лет...,"{'Лечение кариеса': '500—7000', 'Удаление зубо...",{'Тип мед. учреждения': 'стоматологическая кли...,"{'Время работы': ['Пн-Чт, Сб-Вс: 10:00—20:00 и...",none
1,https://zoon.ru/msk/medical/stomatologicheskay...,Никор,"Москва, к1204, Зеленоград",89680404512,"Пн-Пт: 08:00—20:00\nпн-пт с 08:00 до 20:00, об...",none,none,none,none,none,...,4.3,24 отзыва,"Ищете стоматологию в Зеленограде, чтобы решить...",none,8,{'Ильшат Ильгизарович Магфурзянов ': ['Стаж 17...,"{'Лечение кариеса': '≈ 4000', 'Удаление зубов'...",{'Тип мед. учреждения': 'стоматологическая кли...,"{'Адреса сети': ['Вся сеть Никор, группа компа...",none
2,https://zoon.ru/msk/medical/klinika_aksis_v_ze...,Клиника Аксис,"Москва, к1130, Зеленоград",89261847920,Пн-Пт: 08:00—20:00; Сб-Вс: 09:00—18:00\nАксис ...,none,none,none,none,none,...,4.0,182 отзыва,Клиника «Аксис» является многопрофильным медиц...,"['Юридическое название: ООО ""АКСИС""']",7,"{'Сергей Анатольевич Гурин ': ['Стаж 14 лет', ...",{'Консультация акушера-гинеколога': '1900—2300...,"{'Тип мед. учреждения': 'детская поликлиника, ...","{'Адреса сети': ['Вся сеть Аксис, сеть многопр...",none


In [31]:
work.dtypes

page_url             object
object_name          object
address              object
phone                object
hours                object
website              object
social_network       object
fb_link              object
inst_link            object
vk_link              object
tw_link              object
clinic_type          object
stom_services        object
clinic_rating       float64
review_number        object
description          object
clinic_info          object
doc_pics              int64
specialists_dict     object
price                object
services             object
contacts             object
soc_links            object
dtype: object

In [32]:
# редактирую столбец с телефонами

work['phone'] = work['phone'].str.replace('tel:+7', '8', regex=False)

In [33]:
# редактирую столбец с ссылками на сайт

work['website'] = work['website'].str.replace("['", "", regex=False)
work['website'] = work['website'].str.replace("']", "", regex=False)

Большая часть ссылок на Зуне через редирект, поэтому собрал их тексты.

Это ошибка: проверять протокол скриптом долго и сложно.

Проще было все же собрать то, что есть и отредактировать регулярками.

In [34]:
# Функция для проверки протокола домена

def check_url(link):
        
    if link['website'] != 'none':
        
        try:
            http.request('GET', 'https://' + link['website'], headers=headers) # сначала пробую https
            return 'https://' + link['website']
            
        except:
            try:
                http.request('GET', 'http://' + link['website'], headers=headers) # если не работает http
                return 'http://' + link['website']
            
            except:
                return link['website']
                    
    else:
        return 'none'

In [ ]:
%%time

work['new'] = work.apply(check_url, axis=1)

# Ушло 1 час 20 минут примерно

In [35]:
# убираю лишнее из столбца "соцсети"

work['social_network'] = work['social_network'].str.replace("['", "", regex=False)
work['social_network'] = work['social_network'].str.replace("']", "", regex=False)

Редиректы на Зуне по одной схеме, поэтому вытащить из них ссылку на аккаунт очень просто

In [36]:
# редактирую ссылки на ФБ

work['fb_link'] = work['fb_link'].str.replace('.*www.facebook.com%2F', 'https://www.facebook.com/', regex=True) # убираю код
work['fb_link'] = work['fb_link'].str.replace('&hash=.*', '', regex=True) # отрезаю хвост

In [37]:
# редактирую ссылки на инстаграм 

work['inst_link'] = work['inst_link'].str.replace('.*www.instagram.com%2F', 'https://www.instagram.com/', regex=True) # убираю код
work['inst_link'] = work['inst_link'].str.replace('%2F&hash=.*', '', regex=True) # отрезаю хвост

In [38]:
# редактирую ссылки на ВК

work['vk_link'] = work['vk_link'].str.replace('.*https%3A%2F%2Fvk.com%2F', 'https://vk.com/', regex=True) # убираю код
work['vk_link'] = work['vk_link'].str.replace('&hash=.*', '', regex=True) # отрезаю хвост

In [39]:
# редактирую ссылки на твиттер (в стомотологии твиттера нет)

#work['tw_link'] = work['tw_link'].str.replace('.*https%3A%2F%2Fvk.com%2F', 'https://vk.com/', regex=True) # убираю код
#work['tw_link'] = work['tw_link'].str.replace('&hash=.*', '', regex=True) # отрезаю хвост

In [40]:
# убираю лишнее из столбца "количество отзывов"

work['review_number'] = work['review_number'].str.replace(' отзывю*', '', regex=True)

In [41]:
work.to_excel('C:/Data/data_stom_new.xlsx', index=False)